<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#План-исследования" data-toc-modified-id="План-исследования-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>План исследования</a></span></li></ul></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

### План исследования
1. Подготовить данные
2. Обучить модель по исходным признакам
3. Проверить значение F1-меры.
4. Если значение недостаточно высокое, внести необходимые корректировки в модель, пока результат не станет удовлетворять.

## Подготовка данных

In [1]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
try:
    data = pd.read_csv('/datasets/Churn.csv')
except:
    raise Exception('Something is wrong. Check the file')

In [3]:
# ознакомимся с данными
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Видим, что пропуски есть только в столбце Tenure, где хранится кол-во лет, которое клиент является пользователем банка. С этим нужно будет разобраться перед тем, как приступать к обучению. Расшифрока других признаков:
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

Целевой признак:
- Exited — факт ухода клиента

In [5]:
# посмотрим, какие данные присутствуют в столбце с пропусками
data.Tenure.value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

Как мы видим, значение 0 присутствует, значит, пропуск скорее всего не у тех, кто и года не провел в сотрудничестве с банком. Заполним пропуски медианой. Дополнительно стоит заметить, что все значения целые, поэтому в типе float надобности нет.

In [6]:
# заполняем строки с пропусками медианой
data.loc[data['Tenure'].isna(), 'Tenure'] = data['Tenure'].median()
# приводим значения к int
data['Tenure'] = data['Tenure'].astype('int64')

In [7]:
# проверяем
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Далее на очереди кодирование. У нас есть категориальные признаки: *CustomerId, Surname, Geography, Gender*. Несмотря на то, что CustomerId является числом, это просто идентификатор клиента. Его присвоение наверняка автоматизировано и не имеет прямого отношения к поведению клиента. То же самое относится к RowNumber, который является лишь индексом строки в датасете. Эти данные мы удалим. То же относится к Surname, попытка предсказывания по фамилии может запутать модель, и слишком «раздует» датасет после кодирования. А вот страну проживания и пол учитывать стоит.

Так как нам нужно перевести только два признака в количественные, и есть уже другие количественные признаки, то воспользуемся OHE.

In [8]:
# удаляем неинформативные признаки
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [9]:
# обозначим единый рандом для всей дальнейшей работы
random_state = 999

In [10]:
# разделим признаки
features = data.drop(['Exited'], axis=1)
target = data['Exited']

In [11]:
# разбиваем данные на обучающую, валидационную и тестовую выборки в соотношении 60:20:20
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=random_state, stratify=target
)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size=0.5, random_state=random_state, stratify=target_test
)

In [12]:
# проверим верность пропорций
print(features_train.shape)
print(target_train.shape)

print(features_valid.shape)
print(target_valid.shape)

print(features_test.shape)
print(target_test.shape)

(6000, 10)
(6000,)
(2000, 10)
(2000,)
(2000, 10)
(2000,)


In [13]:
# запишем функцию для кодирования
def one_hot_encoding(sample):
    sample = sample.reset_index(drop=True)
    enc = OneHotEncoder(drop='first')
    encoder_df = pd.DataFrame(enc.fit_transform(sample[['Gender', 'Geography']]).toarray())
    return sample.join(encoder_df).drop(['Gender', 'Geography'], axis=1)

In [14]:
# закодируем все выборки с признаками
features_train = one_hot_encoding(features_train)
features_valid = one_hot_encoding(features_valid)
features_test = one_hot_encoding(features_test)
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(6000, 11)
(2000, 11)
(2000, 11)


In [15]:
# приведем числовые имена новых столбцов к строкам
features_train.columns = features_train.columns.map(str)
features_valid.columns = features_valid.columns.map(str)
features_test.columns = features_test.columns.map(str)

Есть признаки с разным масштабом, например, баланс на счете и наличие кредитной карты. Масштабируем все значения для дальнейшего обучения модели.

In [16]:
pd.options.mode.chained_assignment = None
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 
           'HasCrCard', 'IsActiveMember', 'EstimatedSalary', '0', '1', '2']
scaler = StandardScaler()
scaler.fit(features_train)
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,0,1,2
0,-0.639786,0.385695,0.730779,0.080803,0.819046,0.637291,0.970437,1.441672,0.917091,-0.572475,-0.571705
1,1.721641,3.140547,1.093479,0.952093,0.819046,0.637291,0.970437,-0.629563,-1.090405,1.746802,-0.571705
2,-0.505144,0.005716,1.456179,0.011581,-0.914997,-1.569141,-1.030464,-0.899666,-1.090405,-0.572475,-0.571705
3,0.810213,-0.469259,0.005380,-0.029760,0.819046,0.637291,-1.030464,-0.052152,-1.090405,-0.572475,-0.571705
4,-1.654786,0.575685,1.456179,1.514668,0.819046,-1.569141,-1.030464,1.130102,0.917091,1.746802,-0.571705


На этом предобработка данных завершена, переходим к исследованию задачи.

## Исследование задачи

Нашим целевым признаком является Exited, и нам потребуется предсказать, уйдет ли клиент от нас (1) или останется (0). По сути нам нужно предсказать один из двух классов, что подходит под задачу классификации. Также можно рассматривать с точки зрения логистической регрессии, поэтому посмотрим и на нее. Проверим сначала баланс классов.

In [17]:
data.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Видим, что в датасете ушедших клиентов намного меньше, чем оставшихся, это скорее всего повлияет на верность прогнозов. Но сначала проверим, потребуется ли компенсировать дисбаланс. Для этого обучим модели и сверим F1. Дополнительно будем рассчитывать AUC-ROC и подберем гиперпараметры для решающего дерева и случайного леса.

In [18]:
# взглянем на логистическую регрессию
model = LogisticRegression(random_state=random_state, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probablities_valid = model.predict_proba(features_valid)[:, 1]
print("F1 for", type(model).__name__, f1_score(target_valid, predicted_valid))
print('AUC-ROC =', roc_auc_score(target_valid, probablities_valid))

F1 for LogisticRegression 0.3309608540925267
AUC-ROC = 0.7608923908759484


In [19]:
# на решающее дерево
for depth in range(1, 16, 1):
    model = DecisionTreeClassifier(random_state=random_state, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)

    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    
    print('max_depth: {0:<2} | F1: {1:<20} | AUC-ROC: {2:<5}'.format(
        depth, f1_score(target_valid, predictions), roc_auc_score(target_valid, probabilities_one_valid)
    ))


max_depth: 1  | F1: 0.0                  | AUC-ROC: 0.6870504483200315
max_depth: 2  | F1: 0.4992743105950653   | AUC-ROC: 0.7394270679377278
max_depth: 3  | F1: 0.48370497427101206  | AUC-ROC: 0.7988071484875358
max_depth: 4  | F1: 0.4982698961937717   | AUC-ROC: 0.8275707582027787
max_depth: 5  | F1: 0.525984251968504    | AUC-ROC: 0.8366872659867968
max_depth: 6  | F1: 0.5736925515055467   | AUC-ROC: 0.8412051064144251
max_depth: 7  | F1: 0.5816485225505443   | AUC-ROC: 0.8261443553552075
max_depth: 8  | F1: 0.6005917159763313   | AUC-ROC: 0.8381098199329984
max_depth: 9  | F1: 0.5493171471927162   | AUC-ROC: 0.8259896295201498
max_depth: 10 | F1: 0.5470588235294118   | AUC-ROC: 0.7983244962557887
max_depth: 11 | F1: 0.5399449035812672   | AUC-ROC: 0.7690128029855158
max_depth: 12 | F1: 0.5456989247311829   | AUC-ROC: 0.7385033316090255
max_depth: 13 | F1: 0.521164021164021    | AUC-ROC: 0.7229606980983347
max_depth: 14 | F1: 0.5445161290322581   | AUC-ROC: 0.7190409769435413
max_de

In [20]:
# на случайный лес
for est in range(10, 101, 20):
    for depth in range(10, 20, 1):
        model = RandomForestClassifier(random_state=random_state, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)

        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        print('estimators: {0:<5} | max_depth: {1:<2} | F1: {2:<20} | AUC-ROC: {3:<5}'.format(
        est, depth, f1_score(target_valid, predictions), roc_auc_score(target_valid, probabilities_one_valid)
    ))

estimators: 10    | max_depth: 10 | F1: 0.5507246376811594   | AUC-ROC: 0.8497696817420435
estimators: 10    | max_depth: 11 | F1: 0.5571205007824725   | AUC-ROC: 0.8510313516109962
estimators: 10    | max_depth: 12 | F1: 0.5807453416149068   | AUC-ROC: 0.8480823233816139
estimators: 10    | max_depth: 13 | F1: 0.582701062215478    | AUC-ROC: 0.8329507525371957
estimators: 10    | max_depth: 14 | F1: 0.5753424657534246   | AUC-ROC: 0.833043126170066
estimators: 10    | max_depth: 15 | F1: 0.5688888888888888   | AUC-ROC: 0.8383161210464085
estimators: 10    | max_depth: 16 | F1: 0.5587786259541985   | AUC-ROC: 0.8396247475120702
estimators: 10    | max_depth: 17 | F1: 0.5727411944869831   | AUC-ROC: 0.8430110109370381
estimators: 10    | max_depth: 18 | F1: 0.5748148148148148   | AUC-ROC: 0.8328514508818603
estimators: 10    | max_depth: 19 | F1: 0.5564142194744977   | AUC-ROC: 0.8330646800177358
estimators: 30    | max_depth: 10 | F1: 0.5654281098546041   | AUC-ROC: 0.8640806668144644


Результат у логистической регрессии получился неудовлетворительный, а вот у деревьев удалось достичь заданного критерия. 

Зафисируем гиперпараметры для деревьев. 

DecisionTreeClassifier: 
- max_depth: 8  | F1: 0.6005917159763313   | AUC-ROC: 0.8381098199329984

RandomForestClassifier:
- estimators: 70    | max_depth: 17 | F1: 0.6121212121212122   | AUC-ROC: 0.8620461375504975

Возможно, если сбалансировать классы, то получится улучшить результат. Проверим это.

## Борьба с дисбалансом

In [21]:
# попробуем указать необходимость сбалансировать веса для логистической регрессии и сразу будем измерять AUC-ROC

model = LogisticRegression(random_state=random_state, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1 for", type(model).__name__, f1_score(target_valid, predicted_valid))
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_one_valid))

F1 for LogisticRegression 0.4863436123348017
AUC-ROC: 0.7672969627549513


F1-мера резко подскочила, что подтверждает догадку для этого случая, но все равно недостаточно. Попробуем другой подход: увеличить кол-во исходов ухода, т.е. применим upsampling.

In [22]:
# так как мы пересчитывали индексы для признаков, сделаем то же для целевой выборки
target_train = target_train.reset_index(drop=True)

In [23]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=random_state)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

model = LogisticRegression(random_state=random_state, solver='liblinear', max_iter=1000)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("F1:", f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_one_valid))

F1: 0.4843205574912892
AUC-ROC: 0.7673754803428909


Результат не удовлетворяет. Попробуем наоборот уменьшить кол-во преобладающих исходов.

In [24]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=random_state)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=random_state)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=random_state)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.2)

model = LogisticRegression(random_state=random_state, solver='liblinear', max_iter=1000)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]


print("F1:", f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_one_valid))

F1: 0.47561890472618157
AUC-ROC: 0.7670752660360626


Как мы видим, F1 и AUC-ROC для модели линейной регрессии просто с апсемплингом показывает самый лучший результат. Однако предварительно деревья показывали более лучший результат, поэтому поработаем и с ними. Прежние гиперпараметры могут перестать быть актуальными, поэтому придется посчитать их снова.

In [25]:
# попробуем использовать баланс весов классов
for depth in range(1, 16, 1):
    model = DecisionTreeClassifier(random_state=random_state, max_depth=depth, class_weight='balanced')
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)

    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    print('max_depth: {0:<2} | F1: {1:<20} | AUC-ROC: {2:<5}'.format(
            depth, f1_score(target_valid, predictions), roc_auc_score(target_valid, probabilities_one_valid)
        ))

max_depth: 1  | F1: 0.49096098953377737  | AUC-ROC: 0.6952593851610995
max_depth: 2  | F1: 0.5198889916743756   | AUC-ROC: 0.7491686373041679
max_depth: 3  | F1: 0.5198889916743756   | AUC-ROC: 0.8021125849837423
max_depth: 4  | F1: 0.5273311897106109   | AUC-ROC: 0.8257332926889348
max_depth: 5  | F1: 0.5829244357212953   | AUC-ROC: 0.8467975601044437
max_depth: 6  | F1: 0.5681818181818181   | AUC-ROC: 0.8490037503694944
max_depth: 7  | F1: 0.5770308123249299   | AUC-ROC: 0.8376356352842644
max_depth: 8  | F1: 0.5819592628516005   | AUC-ROC: 0.8294967484481229
max_depth: 9  | F1: 0.5688442211055276   | AUC-ROC: 0.7944586597201695
max_depth: 10 | F1: 0.5417095777548919   | AUC-ROC: 0.7787405163070253
max_depth: 11 | F1: 0.5485232067510548   | AUC-ROC: 0.7718925509902452
max_depth: 12 | F1: 0.5497206703910614   | AUC-ROC: 0.7556717410582324
max_depth: 13 | F1: 0.5450399087799316   | AUC-ROC: 0.7431451066607547
max_depth: 14 | F1: 0.5323741007194244   | AUC-ROC: 0.7160488410188195
max_de

In [26]:
# с апсемплингом
for depth in range(1, 16, 1):
    model = DecisionTreeClassifier(random_state=random_state, max_depth=depth)
    model.fit(features_upsampled, target_upsampled)
    predictions = model.predict(features_valid)

    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    print('max_depth: {0:<2} | F1: {1:<20} | AUC-ROC: {2:<5}'.format(
        depth, f1_score(target_valid, predictions), roc_auc_score(target_valid, probabilities_one_valid)
    ))

max_depth: 1  | F1: 0.49096098953377737  | AUC-ROC: 0.6952593851610995
max_depth: 2  | F1: 0.5198889916743756   | AUC-ROC: 0.7491686373041679
max_depth: 3  | F1: 0.5198889916743756   | AUC-ROC: 0.8021125849837423
max_depth: 4  | F1: 0.5273311897106109   | AUC-ROC: 0.8257332926889348
max_depth: 5  | F1: 0.5834970530451866   | AUC-ROC: 0.847401067839196
max_depth: 6  | F1: 0.5731922398589064   | AUC-ROC: 0.8496773081091733
max_depth: 7  | F1: 0.5815470643056849   | AUC-ROC: 0.8423497696817421
max_depth: 8  | F1: 0.5766283524904214   | AUC-ROC: 0.8316482843137255
max_depth: 9  | F1: 0.5612648221343873   | AUC-ROC: 0.7972629692580551
max_depth: 10 | F1: 0.5465968586387434   | AUC-ROC: 0.7771586178441225
max_depth: 11 | F1: 0.5606060606060606   | AUC-ROC: 0.7750209380234506
max_depth: 12 | F1: 0.5549389567147613   | AUC-ROC: 0.7629122943147109
max_depth: 13 | F1: 0.5496535796766744   | AUC-ROC: 0.7475382426840083
max_depth: 14 | F1: 0.5318892900120337   | AUC-ROC: 0.716091948714159
max_dept

In [27]:
# с даунсемплингом
for depth in range(1, 16, 1):
    model = DecisionTreeClassifier(random_state=random_state, max_depth=depth)
    model.fit(features_downsampled, target_downsampled)
    predictions = model.predict(features_valid)

    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    print('max_depth: {0:<2} | F1: {1:<20} | AUC-ROC: {2:<5}'.format(
        depth, f1_score(target_valid, predictions), roc_auc_score(target_valid, probabilities_one_valid)
    ))

max_depth: 1  | F1: 0.49096098953377737  | AUC-ROC: 0.6952593851610995
max_depth: 2  | F1: 0.5198889916743756   | AUC-ROC: 0.7491686373041679
max_depth: 3  | F1: 0.5198889916743756   | AUC-ROC: 0.8027661284362991
max_depth: 4  | F1: 0.5425531914893617   | AUC-ROC: 0.8264468790028574
max_depth: 5  | F1: 0.5015527950310559   | AUC-ROC: 0.8170440129569417
max_depth: 6  | F1: 0.5389930898321815   | AUC-ROC: 0.8052140297073603
max_depth: 7  | F1: 0.5391621129326047   | AUC-ROC: 0.7973422566262686
max_depth: 8  | F1: 0.5196163905841324   | AUC-ROC: 0.7787790053207213
max_depth: 9  | F1: 0.5314926660914582   | AUC-ROC: 0.7654002241600157
max_depth: 10 | F1: 0.4892561983471075   | AUC-ROC: 0.7483873103261405
max_depth: 11 | F1: 0.4830438378825475   | AUC-ROC: 0.725474030692679
max_depth: 12 | F1: 0.4871155444721529   | AUC-ROC: 0.7180718235786777
max_depth: 13 | F1: 0.48599670510708404  | AUC-ROC: 0.7047569341807074
max_depth: 14 | F1: 0.47588424437299043  | AUC-ROC: 0.7005216031136072
max_dep

Результаты получаются хуже, чем с дисбалансом классов. Посмотрим на случайный лес.

In [28]:
# попробуем использовать баланс весов классов и подобрать гиперпараметры для леса
for est in range(10, 101, 20):
    for depth in range(6, 20, 1):
        model = RandomForestClassifier(random_state=random_state, n_estimators=est, max_depth=depth, class_weight='balanced')
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)

        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        print('estimators: {0:<5} | max_depth: {1:<2} | F1: {2:<20} | AUC-ROC: {3:<5}'.format(
        est, depth, f1_score(target_valid, predictions), roc_auc_score(target_valid, probabilities_one_valid)
    ))

estimators: 10    | max_depth: 6  | F1: 0.6202393906420022   | AUC-ROC: 0.8549649288107204
estimators: 10    | max_depth: 7  | F1: 0.6072607260726073   | AUC-ROC: 0.8624217903241699
estimators: 10    | max_depth: 8  | F1: 0.6086956521739131   | AUC-ROC: 0.8559186865701054
estimators: 10    | max_depth: 9  | F1: 0.6294536817102138   | AUC-ROC: 0.8619730084244752
estimators: 10    | max_depth: 10 | F1: 0.593939393939394    | AUC-ROC: 0.8406524041777514
estimators: 10    | max_depth: 11 | F1: 0.5901639344262295   | AUC-ROC: 0.841214343777712
estimators: 10    | max_depth: 12 | F1: 0.5947368421052632   | AUC-ROC: 0.8447122253423982
estimators: 10    | max_depth: 13 | F1: 0.587431693989071    | AUC-ROC: 0.8412636097152428
estimators: 10    | max_depth: 14 | F1: 0.5665236051502145   | AUC-ROC: 0.8332448086018328
estimators: 10    | max_depth: 15 | F1: 0.5537555228276877   | AUC-ROC: 0.8308022957926888
estimators: 10    | max_depth: 16 | F1: 0.565868263473054    | AUC-ROC: 0.8333325635530593


Видим, что с максимальной глубиной в 9 и 50 оценщиками у случайного леса удается достичь приемлемого результата, который превышает результат обучения с дисбалансом. Именно его и используем в качестве искомой модели.

In [29]:
best_model = RandomForestClassifier(random_state=random_state, n_estimators=50, max_depth=9, class_weight='balanced')
best_model.fit(features_train, target_train)
predictions = best_model.predict(features_valid)

probabilities_valid = best_model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
print("F1", type(model).__name__, f1_score(target_valid, predictions))
print('AUC-ROC:', roc_auc_score(target_valid, probabilities_one_valid))

F1 RandomForestClassifier 0.6327014218009479
AUC-ROC: 0.8670620258153512


Мы получили удовлетворяющую заданию F1-меру, а также посчитали площадь под кривой ошибок, равную 0.87, что на отрезке от 0 до 1 является приемлемым результатом.

## Тестирование модели

Теперь нам нужно проверить работу модели на тестовой выборке, которая была для этого отложена.

In [30]:
# обучаем и проверяем
final = RandomForestClassifier(random_state=random_state, n_estimators=50, max_depth=9, class_weight='balanced')
final.fit(features_train, target_train)
predictions = final.predict(features_test)

probabilities_valid = final.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

print('F1: {0:<20} | AUC-ROC: {1:<5}'.format(
    f1_score(target_test, predictions), roc_auc_score(target_test, probabilities_one_valid)
))

F1: 0.5927710843373494   | AUC-ROC: 0.8403226030344674


На тестовых данных результат получился хуже, но остался в допустимых значениях.

## Вывод

В ходе данного исследования мы пытались построить модель прогноза для события ухода клиента из банка. Эту задачу мы рассматривали как классификацию и использовали логистическую регрессию, решающее дерево и случайный лес. Целевой признак был несбалансирован в датасете, поэтому мы попробовали устранить дисбаланс, баланисируя веса признаков, применяя upsampling и downsampling.

В итоге нужный результат показал случайный лес со сбалансированными весами. Ему были подобраны лучшие гиперпараметры, на которых обучили модель, проверяя F1-меру, объединяющую в себе полноту и точность, а также смотрели площадь под кривой ошибок AUC-ROC. F1 должна была быть не менее 0.59, а AUC-ROC точно больше 0.5, что можно считать границей случайного угадывания. На валидационной выборке значения оказались:

- **F1**: 0.6327014218009479
- **AUC-ROC**: 0.8670620258153512

В конце мы проверили работу на тестовой выборке, получив следующие результаты:

- **F1**: 0.5927710843373494
- **AUC-ROC**: 0.8403226030344674

Исходя из этого, предлагаю использовать данную модель для прогнозирования ухода клиентов.